# `numba` tests

This notebook documents and serves as a scratchpad for exploring `numba`-based acceleration on areal interpolation.

**NOTE** - To be removed/relocated once/if functionality is merged

---

**IMPORTANT**

As of Dec. 17th'20, the multi-core implementation requires the versions in `main` for `pygeos` and `geopandas`. On a working environment with the latest released versions (as the `gds_env:5.0`), this can be achieved by:

```shell
pip install --no-deps git+https://github.com/pygeos/pygeos.git
pip install --no-deps git+https://github.com/geopandas/geopandas.git
```

---

In [1]:
from tobler.area_weighted.area_interpolate import _area_tables_binning, area_tables_binning_numba, area_tables_binning_rtree
import geopandas, pandas

summary = lambda src, tgt: print(
    f"Transfer {src.shape[0]} polygons into {tgt.shape[0]}"
)

## Data setup

- Minimal problem

In [2]:
p = ("https://geographicdata.science/book/_downloads/"\
     "f2341ee89163afe06b42fc5d5ed38060/sandiego_tracts.gpkg")
src = geopandas.read_file(p)

p = ("https://geographicdata.science/book/_downloads/"\
     "d740a1069144baa1302b9561c3d31afe/sd_h3_grid.gpkg")
tgt = geopandas.read_file(p).to_crs(src.crs)

w, s, e, n = tgt.total_bounds
#src = src.cx[w:e, s:n]
summary(src, tgt)

/Users/martin/Git/geopandas/martinfleis/geopandas/geodataframe.py:522: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


Transfer 628 polygons into 644


- Slightly larger problem

In [26]:
# Tracts
p = "https://ndownloader.figshare.com/files/20460645"
src = geopandas.read_file(p)
src = pandas.concat([src]*50)

# Precincts
p = "https://ndownloader.figshare.com/files/20460549"
tgt = geopandas.read_file(p).to_crs(src.crs)
tgt = pandas.concat([tgt]*20)
summary(src, tgt)

Transfer 41100 polygons into 75600


## Correctness

In [3]:
cross2 = area_tables_binning_numba(src, tgt, n_jobs=1)
cross = _area_tables_binning(src, tgt)
(cross != cross2).sum()

Setup: 0.19142699241638184 secs
Buckets+: 2.29705810546875 secs
Intersections: 0.37102293968200684 secs
Conversion: 0.0033349990844726562 secs
Setup: 0.002887725830078125 secs
Buckets: 0.21747422218322754 secs
Intersections: 1.8091838359832764 secs


0

In [5]:
nb = area_tables_binning_numba(src, tgt, n_jobs=1)
rtree = area_tables_binning_rtree(src, tgt, n_jobs=1)
(nb != rtree).sum()

Setup: 0.0034880638122558594 secs
Buckets+: 0.1533811092376709 secs
Intersections: 0.40909600257873535 secs
Conversion: 0.002908945083618164 secs
Setup: 0.000990152359008789 secs
Buckets+: 0.04711103439331055 secs
Intersections: 0.2650790214538574 secs
Conversion: 0.0009160041809082031 secs


0

Setup: 0.0014069080352783203 secs
Buckets+: 0.05430197715759277 secs
Intersections: 0.26604175567626953 secs
Conversion: 0.0008320808410644531 secs


<628x644 sparse matrix of type '<class 'numpy.float32'>'
	with 1913 stored elements in Dictionary Of Keys format>

## Performance

Results with all observations in first dataset:

In [7]:
%timeit cross2 = area_tables_binning_numba(src, tgt)

Setup: 0.0033059120178222656 secs
Buckets+: 0.15469813346862793 secs
Intersections: 3.685073137283325 secs
Conversion: 0.0029549598693847656 secs
Setup: 0.0029697418212890625 secs
Buckets+: 0.14260005950927734 secs
Intersections: 0.2119731903076172 secs
Conversion: 0.002791881561279297 secs
Setup: 0.0023958683013916016 secs
Buckets+: 0.1449720859527588 secs
Intersections: 0.21526503562927246 secs
Conversion: 0.002538919448852539 secs
Setup: 0.002048969268798828 secs
Buckets+: 0.13948297500610352 secs
Intersections: 0.217210054397583 secs
Conversion: 0.0027740001678466797 secs
Setup: 0.002178192138671875 secs
Buckets+: 0.13260293006896973 secs
Intersections: 0.20960402488708496 secs
Conversion: 0.002830982208251953 secs
Setup: 0.001955747604370117 secs
Buckets+: 0.1431879997253418 secs
Intersections: 0.2004399299621582 secs
Conversion: 0.0026292800903320312 secs
Setup: 0.002235889434814453 secs
Buckets+: 0.15158915519714355 secs
Intersections: 0.23699569702148438 secs
Conversion: 0.0031

In [6]:
%timeit cross2 = area_tables_binning_numba(src, tgt, n_jobs=1)

Setup: 0.003650188446044922 secs
Buckets+: 0.15782976150512695 secs
Intersections: 0.43152618408203125 secs
Conversion: 0.002707958221435547 secs
Setup: 0.0025510787963867188 secs
Buckets+: 0.1576528549194336 secs
Intersections: 0.48833417892456055 secs
Conversion: 0.002580881118774414 secs
Setup: 0.0035059452056884766 secs
Buckets+: 0.15597224235534668 secs
Intersections: 0.4255349636077881 secs
Conversion: 0.0026149749755859375 secs
Setup: 0.001977682113647461 secs
Buckets+: 0.13219213485717773 secs
Intersections: 0.41508913040161133 secs
Conversion: 0.002586841583251953 secs
Setup: 0.0022242069244384766 secs
Buckets+: 0.14191484451293945 secs
Intersections: 0.3956260681152344 secs
Conversion: 0.0022780895233154297 secs
Setup: 0.003113985061645508 secs
Buckets+: 0.13562297821044922 secs
Intersections: 0.40630102157592773 secs
Conversion: 0.002393960952758789 secs
Setup: 0.001847982406616211 secs
Buckets+: 0.1367340087890625 secs
Intersections: 0.39743590354919434 secs
Conversion: 0.0

In [10]:
%timeit cross = area_tables_binning_rtree(src, tgt, n_jobs=1)

Setup: 0.0019299983978271484 secs
Buckets+: 0.07179808616638184 secs
Intersections: 0.2904839515686035 secs
Conversion: 0.0007832050323486328 secs
Setup: 0.0008697509765625 secs
Buckets+: 0.05193805694580078 secs
Intersections: 0.28725409507751465 secs
Conversion: 0.0007829666137695312 secs
Setup: 0.0010428428649902344 secs
Buckets+: 0.05407118797302246 secs
Intersections: 0.29442715644836426 secs
Conversion: 0.0007879734039306641 secs
Setup: 0.0008792877197265625 secs
Buckets+: 0.055357933044433594 secs
Intersections: 0.33969998359680176 secs
Conversion: 0.0010650157928466797 secs
Setup: 0.002268075942993164 secs
Buckets+: 0.06393909454345703 secs
Intersections: 0.3237020969390869 secs
Conversion: 0.0007758140563964844 secs
Setup: 0.0008871555328369141 secs
Buckets+: 0.05243802070617676 secs
Intersections: 0.3357710838317871 secs
Conversion: 0.0010869503021240234 secs
Setup: 0.0012958049774169922 secs
Buckets+: 0.06059598922729492 secs
Intersections: 0.33737993240356445 secs
Conversio

In [8]:
%timeit cross = _area_tables_binning(src, tgt)

Setup: 0.0047380924224853516 secs
Buckets: 0.27542591094970703 secs
Intersections: 1.9011662006378174 secs
Setup: 0.0019876956939697266 secs
Buckets: 0.2174520492553711 secs
Intersections: 1.9076390266418457 secs
Setup: 0.002991199493408203 secs
Buckets: 0.22337889671325684 secs
Intersections: 2.037278890609741 secs
Setup: 0.0020818710327148438 secs
Buckets: 0.3154489994049072 secs
Intersections: 2.575305938720703 secs
Setup: 0.0018458366394042969 secs
Buckets: 0.26967811584472656 secs
Intersections: 2.398831844329834 secs
Setup: 0.0020449161529541016 secs
Buckets: 0.27692317962646484 secs
Intersections: 2.3819448947906494 secs
Setup: 0.0022649765014648438 secs
Buckets: 0.2858550548553467 secs
Intersections: 2.3310320377349854 secs
Setup: 0.0020520687103271484 secs
Buckets: 0.2722609043121338 secs
Intersections: 2.5641541481018066 secs
2.58 s ± 263 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


---

Results with second dataset:

In [ ]:
%time cross2 = area_tables_binning_numba(src, tgt)

In [ ]:
%time cross2 = area_tables_binning_numba(src, tgt, n_jobs=1)

In [ ]:
%time cross = _area_tables_binning(src, tgt)

---

Results with second dataset, flipped:

In [13]:
%time cross2 = area_tables_binning_numba(tgt, src)

Setup: 0.0039517879486083984 secs
Buckets+: 2.526702880859375 secs
Intersections: 1.6080853939056396 secs
Conversion: 0.012014150619506836 secs
CPU times: user 3.38 s, sys: 86.9 ms, total: 3.47 s
Wall time: 4.15 s


In [14]:
%time cross2 = area_tables_binning_numba(tgt, src, n_jobs=1)

Setup: 0.004775524139404297 secs
Buckets+: 2.515500068664551 secs
Intersections: 4.918679475784302 secs
Conversion: 0.01279139518737793 secs
CPU times: user 7.43 s, sys: 37.2 ms, total: 7.47 s
Wall time: 7.46 s


In [15]:
%time cross = _area_tables_binning(tgt, src)

Setup: 0.003404378890991211 secs
Buckets: 0.7955982685089111 secs
Intersections: 8.69243049621582 secs
CPU times: user 9.49 s, sys: 0 ns, total: 9.49 s
Wall time: 9.49 s


To do:

- [X] Paralellise `pygeos` operations (`parall_exec`)
- [ ] Type inputs (explicitly like [here](https://github.com/pysal/esda/blob/master/esda/crand.py#L309))
- [ ] Document
- [ ] Add tests